In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, searches, regularizers, evaluators, trainers, datasets, distributions

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

In [4]:
# k
ks = [5, 10, 50]

score_function_dict = {
    #"Recall"       : evaluators.recall
    #"Unpopularity" : evaluators.unpopularity,
    #"Unpopularity2": evaluators.unpopularity2,
    #"Unpopularity3": evaluators.unpopularity3,
    #"F1-score"     : evaluators.f1_score,
    #"F1-score2"    : evaluators.f1_score2,
    #"F1-score3"    : evaluators.f1_score3
    "my_metric1"   : evaluators.my_metric1,
    "my_metric2"   : evaluators.my_metric2
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)
# coverage = evaluators.CoverageEvaluator(test_set, ks)
# hubness = evaluators.HubnessEvaluator(test_set, ks)

In [5]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [8]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 30
valid_per_epoch = 50
n_sample = 30
bias = 0.5
n_item_sample = 30
n_user_sample = 30
no_progressbar = False

search_range = 30

# models
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)

# distributiuons
gaussian = distributions.Gaussian()
gamma = distributions.Gamma()

# search
knn = searches.NearestNeighborhood(model)
mp = searches.MutualProximity(model, gamma, n_item_sample, n_user_sample, bias)
ndmp = searches.NoDistinctionMutualProximity(model, gamma, n_sample, bias)

# learning late optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [9]:
trainer.fit(n_batch, n_epoch)

epoch30 avg_loss:0.255: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:01<00:00, 150.31it/s]


In [10]:
trainer.valid(knn, userwise)
re1 = trainer.valid_scores.copy()
display(re1)
trainer.valid(mp, userwise)
re2 = trainer.valid_scores.copy()
display(re2)
trainer.valid(ndmp, userwise)
re3 = trainer.valid_scores.copy()
display(re3)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:00<00:00, 1330.81it/s]


,my_metric1@5,my_metric2@5,my_metric1@10,my_metric2@10,my_metric1@50,my_metric2@50
0,6.44358,0.546221,6.294985,0.451664,4.321862,0.200244


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:04<00:00, 230.55it/s]


,my_metric1@5,my_metric2@5,my_metric1@10,my_metric2@10,my_metric1@50,my_metric2@50
0,8.366115,0.433352,7.837173,0.373417,4.908518,0.186093


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:05<00:00, 166.35it/s]


,my_metric1@5,my_metric2@5,my_metric1@10,my_metric2@10,my_metric1@50,my_metric2@50
0,6.953495,0.51091,6.740889,0.432875,4.482289,0.198952
